# Computer Vision Training YM

## Exercise format
The sections of code that you will have to fill in yourself are marked by blocks starting with ##### and ending with #=======.

#>>(notes)lines provide hints for the problem and #- lines are stand-ins for statements you need to fill in. You'll often find variables in the format ```variable = None```, these can be variables or objects where I've provided the name so they'll be consistent with code later in the exercise. 

These are just guidelines and if you have a different idea of how to approach the problem feel free to deviate. Try to think of what conceptually needs to happen to solve the problem and then implement it. Don't stay with one problem for too long if you get stuck. Instead, just look at the answers. Sometimes these problems come down to knowing some specific syntax and become a lot harder if you don't. It is more important that you get a feel for some of these concepts so have a starting point if you have to work with them for a project than that you solve all of on your own.

## 1: Introduction to Convnets with pytorch 
In this section we will build a simple convolutional model and train it to classify images from the CIFAR-10 dataset. The CIFAR-10 dataset is one of the first large scale image datasets, but the images are very small (32x32). All convolutional models will do well on this if they have enough layers.

#### Download the data using the torchvision library
Pytorch has some build in libraries to download certain datasets, the CIFAR-10 dataset is one of them. 

In [7]:
# convnet classification task on 
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np

transform = transforms.Compose([
    # you can add other transformations in this list if you want.
    transforms.ToTensor()
])

#################
# >> make cifar10 test and train dataset objects
cifar10_train_dataset = None
cifar10_test_dataset = None
#================

Files already downloaded and verified
Files already downloaded and verified


#### Make dataloaders from the datasets
Wrap the dataset classes in a pytorch dataloader class. This will allow us to feed the data to the model in a easy and controllable way.

In [ ]:
# adjust batch size to fit gpu space AND for finetuning the training process, you'll have to do this for all different exercises.
# batch size is not an insignificant factor in the training of convnets (or any neural networks for that matter)
batch_size = 32

#################
# >> make train and test dataloader objects
train_loader = None
test_loader = None
#================


# the dataset class should contain the mapping of label to label_idx
label_to_classname = {v: k for k, v in cifar10_test_dataset.class_to_idx.items()}

# show single sample
for batch, target in test_loader:
    sample = batch[0].permute(1, 2, 0)
    t_label = target[0].item()
    print(f"target label    : {label_to_classname[t_label]}")
    print(f"shape of tensors: batch={batch.shape}, target={target.shape}")
    plt.imshow(sample.numpy())
    break

#### Build your convolutional model
Use 2d Convolutional layers and ReLU layers to construct a simple neural network. You will also need a linear layer (also called fully connected layer) at the end and perhaps some dropout layers.

In [ ]:
class VerySimpleNet(nn.Module):
    def __init__(self, n_classes):
        super(VerySimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.relu1 = nn.ReLU()
        
        #################
        # >> add a few layers, I suggest alternating conv2d and relu layers while increasing the amount of filters 
        
        self.fc = None
        #================


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        #################
        # >> call layers in the forward pass
        
        #================
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

#### Training and testing the model
We'll now define basic training and testing routines. In pytorch you'll have to specify which optimizer, loss and scheduler (if you want to use one) you want to use and put the statements in the right spots.

In [ ]:
# I've written the train and test methods for you, have a look at them and see if you understand what they do.
def train(model, device, train_loader, optimizer, criterion, epoch, log_interval=5, dry_run=False):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #################
        # data and target to gpu
        #-
        
        # reset gradients of previous iteration
        #-
        
        # forward pass of model
        #-
        
        # calculate loss
        #-
        
        # calculate gradients
        #-
        
        # apply optimizer based on gradients
        #-
        #================
        # log the loss
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            #################
            # data to gpu
            #- 
            
            # data through model
            #- 
            # output = ...
            # calculate loss (only for logging, we're not going to use it for backpropagation)
            #- 
            
            # add loss to total loss
            test_loss += loss
            
            #================
            # calculate metric
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# some parameters
device = "cuda:0"
lr = 0.001
gamma = 0.7
epochs = 5
model = VerySimpleNet(10).to(device) # transfer model to GPU

#################
# >> define optimizer, loss function (criterion) and scheduler
optimizer = None
criterion = None
scheduler = None
#================

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval=100)
    test(model, device, criterion, test_loader)
    scheduler.step()

#### Inspecting the results
Lets look at some classification examples from the test set, how does your model perform?

In [ ]:
for batch, target in test_loader:
    sample = batch[0].permute(1, 2, 0)
    output = model(batch.to(device))
    p_label = torch.argmax(output[0]).item()
    t_label = target[0].item()
    print(f"target label: {label_to_classname[t_label]}, predicted label: {label_to_classname[p_label]}")
    plt.imshow(sample.numpy())
    
    break

## Understanding Convnets - Architecture, Backbones & Layers
Backbones are large (often pretrained) neural networks that serve as a way to extract features that are useful for almost all types of problems.
The general idea of a backbone is to have a model that is very good at identifying high level features in an image. 
The easiest way to understand this is to think of images as collections of shapes. A face is just a collection of circles (such as the eyes) and curvy lines.
This means that if we already have a model that can detect all these components the step to the full combination of those components (such as a face) is a lot easier!
In practice most pretrained backbones will already have concepts such as faces embedded into the layers. Additionally, lines and circles are actually pretty basic features and a large pretrained backbone will contain much more complex concepts.

### Revisiting CIFAR-10 
Now we know about backbones, let's use one on the CIFAR-10 dataset. For this we'll need to download a pretrained model and adjust the number of classes. We'll pick the resnext50 backbone model, a variant of the resnet style architecture (see, https://pytorch.org/hub/pytorch_vision_resnext/).

In [ ]:
#################
# >> import a pretrained backbone from the torchvision.models library

pretrained_model = None
#================

# you can always print a model to see the full structure or even the partial structure if you select it.
print(pretrained_model.layer1)

#### Changing a backbone
We now want to change the backbone so it fits our 10 class classification problem. There are a few ways to do this and here we will look at both of the methods.

1. The first (and easiest) way is to change the model is to just swap out the last layer. In this case the original model was already a classification model and we are just changing the number of output classes in the last layer to fit our problem. In this case this is also the most 'correct' way of doing it.
2. The second way to adjust the model is to wrap it in a new model class. This method is useful in more complicated scenarios where you want to change more than just the number of outputs. For example if we wanted to use the backbone as the basis for a segmentation model. Now before you ask, yes wrapping the backbone like this preserves the last layer that would be replaced in the other example, luckily this does not effect performance (only memory usage).

1: replacing layers

In [ ]:
num_classes = 10
#################
# >> change the last layer of the model, (run the training cell further below to see if it worked):

#================

print(pretrained_model)

2: wrapping modules

In [ ]:
# now let's wrap our model in a pytorch module instead, this is syntactically the same as adding a layer to a regular network.

class WrappedResnext(nn.Module):
    def __init__(self, n_classes, backbone):
        super(WrappedResnext, self).__init__()
        #################
        self.backbone = None
        self.fc2 = None
        #================

    def forward(self, x):
        #################
        # >> do forward pass
        
        #================
        return x

#################
# >> import unchanged model again
pretrained_model = None
wrapped_model = None
#================

print(wrapped_model)

#### Test your adjusted models by running the cell below
Now that we've adjusted our model for our problem we can try it out. Try both ways

In [ ]:
device = "cuda:0"
gamma = 0.7
epochs = 10
learning_rate = 0.001

wrapped_model = wrapped_model.to(device)

#################
# >> define optimizer, loss function (criterion) and scheduler again. You've done this before, I'm just making you do it again.
optimizer = None
criterion = None
scheduler = None
#================

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval=100)
    test(model, device, criterion, test_loader)
    scheduler.step()

#### Inspecting the results
And? does the model perform better? can you even tell without training for a (very) long time?
try looking at some examples.

In [ ]:
# If you want to see some samples
for batch, target in test_loader:
    sample = batch[0].permute(1, 2, 0)
    output = model(batch.to(device))
    p_label = torch.argmax(output[0]).item()
    t_label = target[0].item()
    print(f"target label: {label_to_classname[t_label]}, predicted label: {label_to_classname[p_label]}")
    plt.imshow(sample.numpy())
    
    break

# Before you start this section!
We will be using another fairly large dataset in this section so turn on the download by running the imports and the *Testing the model* section 

#### Utility: clearing GPU memory
when working with pytorch or any deep learning framework you will likely get errors concerning the GPU memory. Take a look at the code below, running this method (or something similar) can help clearing the GPU memory if this becomes a problem.

In [ ]:
torch.cuda.empty_cache()

import gc # gc = garbage collection
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
            if obj.size() == torch.Size([4, 21, 568, 568]):
                del obj
            if obj.size() == torch.Size([4, 3, 568, 568]):
                del obj
            if obj.size() == torch.Size([4, 1, 568, 568]):
                del obj
    except:
        pass

# 2: Understanding Convnets - Model architecture
In this section we'll look at Unet, a classic (compared to more recent models) deep learning segmentation model that introduced the concept of encoder-decoder architecture. It is also fairly easy to understand. It consists of two parts, the encoder and the decoder. The Encoder has the same job as the backbone has in the previous section: Its purpose is to extract features (you can thing of the as classifications or objects) from the image, that will then be used in the second half of the model (the decoder) to make the segmentation, (place those objects in the right place with the right boundries). In this sense, the Decoder does the same thing as the single layer of linear nodes in the backbone example. It uses the features supplied by the encoder to make a classification, just for all pixels in the image instead of the entire image. 

So thats the general idea, but what are some of the other interesting parts of the model? 
1. As we can see in the image below, the shape of the layers gets deeper in filters but smaller in width/height.
This is done to allow the model to learn larger more complex concepts. As the size of the convolutional filters stays the same throughout the model (generally kernel size is always 3x3), a larger object like a car would never fit in those 3 pixels. By downsizing the output after each layer, a full object CAN be represented in that 3x3 grid of pictures. 
This is because filters specialize, a certain filter in the 4th layer of the model might only respond to a specific pattern that was found in the previous layer. That pattern is again already an abstraction upon the previous input etc. etc. until you reach the first layer where only lines and squiggles are detected.

2. What are those grey lines jumping from the enconder part to the decoder part? As you might have suspected this is just the output of that layer being saved and copied to be concatenated to the decoder at the mirrored layer later. This is done because a lot of spatial detail is lost in the process of downsampling. By adding the higher resolution data from the downsampling process this effect is somewhat mitigated as the network is able to use this data to make more precise object boundaries. 

![alt text](https://miro.medium.com/max/2824/1*f7YOaE4TWubwaFF7Z1fzNw.png "Logo Title Text 1")


#### Building the segmentation model
I've defined some functions to get you started, try to complete the model based on the architecture shown above. you can use a linear layer at the end but you don't have to.

In [ ]:
# imports so you don't have to scroll up when you get an OOM error
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## From classification model to segmentation model

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   

class UNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()  
        #################
        # >> finish the encoder and then make the decoder
        # encoder
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = None
        self.dconv_down4 = None

        # up
        self.maxpool = nn.MaxPool2d(2)
        
        # down
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        # decoder
        # >> add layers for the decoder yourself 
        
        
        # >> final layer
        self.final_layer = None
        #================
        
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        #################
        # >> finish the encoder forward pass
        
        
        # >> add the decoder forward pass
        
        #================
        
        out = self.final_layer(x)
        return out

#### Testing the model 
Lets try out our new model on another pytorch/torchvision dataset. We'll use the SBD dataset, another dataset that can be downloaded using the torchvision dataset library.
This dataset supplies full segmentations instead of just classes. Its a little trickier to use so I've completed this section for you, feel free to look around though.

In [ ]:
from PIL import Image

# custom compose object to transform the pil images to tensors in the right format
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image = t(image)
            if type(t) != transforms.Normalize:
                target = t(target)
            
        return image, target

transform = []
transform.append(transforms.Resize((568, 568), interpolation=Image.NEAREST))
transform.append(transforms.ToTensor())
transform.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]))
transform = Compose(transform)

# sadly this dataset didnt have the classes attribute so I had to add them manually.
label_to_classname = {
    0: "background",
    1: "aeroplane",
    2: "bicycle",
    3: "bird",
    4: "boat",
    5: "bottle",
    6: "bus",
    7: "car",
    8: "cat",
    9: "chair",
    10: "cow",
    11: "diningtable",
    12: "dog",
    13: "horse",
    14: "motorbike",
    15: "person",
    16: "pottedplant",
    17: "sheep",
    18: "sofa",
    19: "train",
    20: "tvmonitor"
}

# this dataset is 1.4 Gigabyte so be patient. 
sbd_dataset = torchvision.datasets.SBDataset(root="./sbd", download=False, mode="segmentation", image_set='train', transforms=transform)

# pick the right batch_size, generally you would want to use at least 16 for any convnet training, 
# but often this will be impossible due to the size on smaller gpus, we're really only trying stuff out so you can use a smaller size if needed here
batch_size = 4

train_loader = torch.utils.data.DataLoader(sbd_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=5)

#### Inspecting the data
Run this a few times to look at some examples

In [ ]:
for image_batch, target_batch in train_loader:
    image = image_batch[0]
    target = target_batch[0]
    
    image_cl = image.permute(1, 2, 0)
    target_cl = target.permute(1, 2, 0)
    
    fig, axs = plt.subplots(1, 2, figsize=(20,10))
    
    # We are plotting normalized images here, you could turn denormalize them with the values in the transform operation, but for the purpose of seeing
    #the content of the image this is just fine
    axs[0].imshow(image_cl.numpy())
    
    # naive approach to muticlass plotting, classes are assigned different colors due to the values being different, but is not consistent between images
    axs[1].imshow(target_cl.numpy())
    print(f"classes: {[label_to_classname[c] for c in np.unique(target_cl.numpy()*255)]}")
    
    break

### Training the model
Training the segmentation model is basically the same as for any neural network. Just plug in the data and let the optimizer do the work. Fitting this model is a pretty heavy task and it might take too long, I've added an overfit function that will just overfit the model on 1 image, things like this are a good way of testing whether the model is capable of processing this type of data. For the model to perfectly overfit an image the whole training routine has to work.  

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, log_interval=5, dry_run=False, overfitrun=False):
    model.train()
    losses = []
    n_iters = 0
    overfitdata = None
    overfittarget = None
    for batch_idx, (data, target) in enumerate(train_loader):
        if overfitrun:
            if batch_idx == 0:
                overfitdata = data 
                overfittarget = target
            else:
                data = overfitdata
                target = overfittarget
            
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        
        target = target.reshape(batch_size, 568, 568) * 255
        target = target.long()
        loss = criterion(output, target)
        losses.append(loss.item())
        n_iters += 1
        
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t average Loss (last 500): {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses[-500:])))
            if dry_run:
                break

# parameters
device = "cuda:0"
epochs = 10
learning_rate = 0.001
gamma = 0.7

#################
# >> this time write the training code yourself (its still more or less the same as before)) 
# >> make the model

# >> define optimizer, loss function and scheduler

# >> do the train and test step for n epochs

    
#================

#### You can save the model if you want or load one I pretrained
If your model is taking a while to train, you can copy the example model code and import these pretrained weights to see some results.

In [ ]:
# save model 
# torch.save(segmentation_model.state_dict(), "segmentation_model.pt")

# load model
# segmentation_model = UNet(sbd_dataset.num_classes + 1)
# segmentation_model.load_state_dict(torch.load("segmentation_model.pt"))
# segmentation_model.eval()
# segmentation_model = segmentation_model.to(device)
print("done!")

#### Inspect the results
You can see the results of your training here. keep in mind however that it takes a long time to train a large model like this. You won't get good looking results unless you leave it running for a while. Try the pretrained backbone (look in the answers to find my implementation that fits it) for a working(-ish) example.

In [ ]:
for image_batch, target_batch in train_loader:
    #################
    # >> extract the first image in the batch and its target
    
    # >> switch channel orders for plotting
    
    # >> pass the batch through the model and get the predictions
    
    # >> get the predictions for the first image and transform them for plotting 
    
    #================
    fig, axs = plt.subplots(1, 3, figsize=(20,10))
    axs[0].imshow(image_cl.numpy())
    
    # naive approach to muticlass plotting, classes are assigned different colors due to the values being different, but is not consistent between images
    axs[1].imshow(target_cl.numpy())
    axs[2].imshow(pred.numpy())
    print(f"classes: {[label_to_classname[c] for c in np.unique(target_cl.numpy()*255)]}")
    
    break

# 3: Understanding Convnets - Filter Visualization 

An interesting area of research within convolutional neural networks is the interpretability. As you will likely know, neural networks are mostly black box solutions. It is hard to get an idea of why the network performs the way it does. There are several ways in which more insight can be gained from the model, but most of them focus on various ways of tracking the activation of the model on a certain image. Doing this for a large amount of images gives insight into what parts of the model respond to specific stimuly. This process is somewhat similar to how human or animal brains can be studied as well, if you are shown multiple similar pictures, most likely the same area of the brain will be used. 

Using neural networks however, we can do more than just track the activation throught the network. Neural networks, although they are large black boxes, are deterministic. This means that we always get the same output for the same image, but more interesting, this means we can make small adjustments to the input image and by doing so map the internal logic of the network! 

In this example we will apply a process called deep-dreaming (https://en.wikipedia.org/wiki/DeepDream) to see inside a neural network. More specifically we will change the optimization goal to visualize the convolutional filters. We will do so by inputting random noise and adjusting that noise to get a higher activation of a specific filter/layer. Adjusting values to get a higher activation? does that sound familiar? Well it should because this method uses the same backpropagation algorithm as regular training just with a different target!

## Preparing our model
For this task we will use a pretrained model. This is because a pretrained model will already have well trained filters that look for very specific patterns. If we were to use a untrained model we might not get good visualizations at all!

In [ ]:
# imports
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import requests
import numpy as np
import torchvision.models as models
from io import BytesIO
from PIL import Image

In [ ]:
# lets use the same backbone model as before

#################
# >> import pretrained model
model = None

#================

# set in eval mode to avoid adjusting the weights
model.eval()
print("done")

In [ ]:
# have a look at the layers & modules in the model and pick one as optimization target
# list(resnext50.modules())
model.layer2[1].conv1

#### Retrieving intermediate results
To do the optimization we need the output of a specific layer/module in the model. We can do this in pytorch by making use of a Hook. A hook will be attached to a specific location in the model and will save automatically save what we want when the models forward function is called. 

In [ ]:
class Hook():
    def __init__(self, module, backward=False):
        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)

    def hook_fn(self, module, input, output):
        self.input = input
        self.output = output
        
    def close(self):
        self.hook.remove()

#### Getting gradients for optimization
Instead of optimizing using the loss gained by comparing the output to the target label, our loss will just be the activation of the layer that we set as target. Additionally we will be trying to get the loss as high as possible instead of low like in a regular training setup, this is called gradient ascent. 

In [ ]:
#Function to make gradients calculations from the output channels of the target layer  
def get_gradients(model_input, model, layer, filter_idx):     
    model_input = model_input.unsqueeze(0).cuda() # some reshaping
    
    #################
    # >> fill in these one line statements at the #-
    
    # we want to get gradients from the forward pass so make sure the inpute data generates gradients
    #-
    
    # discard any previous gradients
    #-
    
    # apply the hook we made earlier to the target layer
    #-
    
    # do the forward pass, we won't actually use the final output
    #-
    
    # get the loss by retrieving the output saved by the hook.
    # we will take the norm of the output because we want extreme values in both directions, positive AND negative.
    # don't forget that the data is in batched format, even though we're only supplying one example 
    #-
    
    # use the loss to calculate the gradients
    #-
    
    # return the gradients we just calculated (and reshape data)
    return None
    #================

#### Gradient Ascent
Now that we have a way to get specific outputs from inside the model and a way to calculate gradients for our optimization target, we can write the full deep dream function.
In this function we will prepare the image, perform the gradient ascent and return the final output.

In [ ]:
# denormalization image transform, used to give the image the right colors again.
denorm = transforms.Compose([transforms.Normalize(mean = [ 0., 0., 0. ], std = [ 1/0.229, 1/0.224, 1/0.225 ]),                 
                             transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ], std = [ 1., 1., 1. ]),                                                     
                            ])

def dream(image, model, layer, iterations, lr, filter_idx):
    """Deep dream an image using given model and parameters."""
    #################
    # >> fill in these one line statements at the #-
    
    # convert image to tensor
    #-
    
    # remove additional channels if it's present (pngs will have a 4th transparancy channel)
    #-
    
    # normalize the image
    image_tensor = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image_tensor).cuda()
    
    # deep dream the image
    for i in range(iterations):
        # get gradients 
        gradients = get_gradients(image_tensor, model, layer, filter_idx)
        
        # add gradients to image to morph the image in a direction that causes a higher activation, we'll add a learning rate parameter to control the effect
        #-
        
    #================
    # get the final image from gpu
    img_out = image_tensor.detach().cpu()
    
    # denormalize
    img_out = denorm(img_out)
    
    # do some reshaping, conversion
    img_out_np = img_out.numpy().transpose(1,2,0)
    img_out_np = np.clip(img_out_np, 0, 1)
    img_out_pil = Image.fromarray(np.uint8(img_out_np * 255))
    return img_out_pil

#### Deep Dreaming 
Now that we have all our functions, let's try them out on a bunch of images!

In [ ]:
device = "cuda:0"

# Get an input image, I've used weblinks here but you can upload your own as well, you could even try using random noise!
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Kladsk%C3%A1_forrest.jpg/1024px-Kladsk%C3%A1_forrest.jpg'
# url = 'https://www.marineterrein.nl/wp-content/uploads/2019/02/IMG_8815-830x466.jpg'
# url = 'https://youngmavericks.com/img/contact_location.png'

response = requests.get(url)
img = Image.open(BytesIO(response.content))

# might want to resize if they're very large, it would take a while to do the whole deep dream
orig_size = np.array(img.size)
# new_size = np.array(img.size)*0.5

#################

# make sure the model is on the gpu
#-

# pick a target layer
#-

# set some parameters
filter_idx = None
learning_rate = None
iterations = None

# call the deep dream function on an image and get the deep dreamed image
# -

#================

# resize to original if needed 
img = img.resize(orig_size)

# plot
fig = plt.figure(figsize = (20 , 20))
plt.imshow(img)

# Convnets for spatially related non-image data
### Downloading the data.
The data for this section is quite large so before going any further, start the download :). Download the data here. https://drive.google.com/file/d/1r_SUJpfz3qX0j6ZwmFwCBTEHw7EE-q4l/view?usp=sharing & https://drive.google.com/file/d/1GO6Stq_eRsJGaQL8KVWoex1A2vcX4D87/view?usp=sharing

dataset site:
http://machine-listening.eecs.qmul.ac.uk/bird-audio-detection-challenge/#downloads

### Overview 
Convnets are typically associated with image classification tasks, however any spatially related data can be classified with convnets. In this example we will see how it possible to classify audio data with convnets. More specifically, we will try to classify short audio samples as contains birdsong or not. 
To do this, we'll first have to convert audio data into something that can be processed using a neural net. Spectograms (https://en.wikipedia.org/wiki/Spectrogram) are images that show the frequencies present in sound data over a period of time. These images will then be fed to the convnet together with the label.

### Making the spectograms
To make the spectograms we will make use of the scipy signal processing library, this is pretty straightforward.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import wavfile

#################

# Read the wav file (stereo)
# - 

# convert to mono if needed and reshape, the bird data should already be mono
#-

# crop to 10 seconds (44kHz audio), it should be 10s already anyway
#-

#make the spectogram using the signal library
#- frequencies, times, spectrogram = ... 

#================
# draw the image
plt.pcolormesh(times, frequencies, np.log(spectrogram), shading='auto')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
# some parameters
#################
basedir = "path/to/spectogram/save/folder/" # i suggest making a spectogram/ folder to save the images in 
filenames = glob("/path/to/wav/folder/*.wav")
#================

os.makedirs(basedir, exist_ok=True)

#### Convert all samples to spectograms
We will now use this sample process to convert all audio samples to spectograms, to speed it up, lets use the python multiprocessing pool function.

In [ ]:
import os
import sys 
from tqdm import tqdm
from multiprocessing import Pool
import matplotlib.pyplot as plt
import math

def make_spectogram(filename):
    #################

    # Read the wav file (stereo)
    # - 

    # convert to mono if needed and reshape, the bird data should already be mono
    #-

    # crop to 10 seconds (44kHz audio), it should be 10s already anyway
    #-

    #make the spectogram using the signal library
    #- frequencies, times, spectrogram = ... 

    #================

    # To make a figure without the frame :
    fig = plt.figure(frameon=False)
    w = 10
    h = 5
    fig.set_size_inches(w,h)

    # To make the content fill the whole figure
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    # draw the image
    plt.pcolormesh(times, frequencies, np.log(spectrogram), shading='auto')
    dpi = 50
    savename = os.path.join(basedir, os.path.basename(filename).split(".")[0] + ".jpg")
    fig.savefig(savename, dpi=dpi)
    plt.close(fig)

# turn of plot generation
plt.ioff()

# this will take a few minutes.
p = Pool(4) # adjust number of cores to desired amount or use os.cpu_count()
with p:
    p.map(make_spectogram, filenames)

## Make the pytorch dataset class

### Read the labels
Labels are given in a csv file, each row contains the name of the sample and the classification (bird or notbird), we can read them using pandas and split them up into train/test sets.

In [ ]:
# parameters
csv_file = '/home/parting/Downloads/warblrb10k_public/warblrb10k_public_metadata.csv'
device = "cuda:0"
train_test_split = 0.8

#################
# read csv with pandas
df = None

# split up the dataframe into train and test
train_df = None
test_df = None

#================
print(f"length of trainset: {len(train_df)}, length of test set: {len(test_df)}")

# example
print("example:")
print(f"    spectogram with key: ({'759808e5-f824-401e-9058'}) has class: ({int(df.loc['759808e5-f824-401e-9058'][1])})")

#### Pytorch dataset class
To load and preprocess all the spectograms we will need a new pytorch dataset class, we will then wrap this dataset class in a dataloader.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
import pandas as pd
import numpy as np
import torch.optim as optim
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR

#################
class BirdDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # setup all the object attributes


    def __len__(self):
        # return the amount of samples
        return None

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        #- read the image
        
        # -get the label

        #- normalize the image

        # return the image, target and image name (for debuggin later)
        return image, label, img_name

#================
# some standard transforms
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#################
# make the train and test datasets
birddataset_train = None
birddataset_test = None

batch_size = 32

# wrap them in dataloaders
train_loader = None
test_loader = None

#================
# try out the dataset
sample, label, fname = birddataset_train.__getitem__(0)
print(f"sample shape: {sample.shape}, label: {label}, filepath: {fname}")

#### Training and testing the model
We'll use basically the same code that we used to train on the cifar-10 and segmentation datasets, so I'm not going to give you a lot of help here. If you're curious, the original paper achieved about 88% accuracy on this dataset, how close can you get ;)?

In [ ]:
#################
def train(model, device, train_loader, optimizer, criterion, epoch, log_interval=5, dry_run=False):
    model.train()
    # do training


def test(model, device, criterion, test_loader):
    model.eval()
    # do testing
    

lr = 0.001
gamma = 0.7
epochs = 5

# prepare a pretrained model to train and use.


# define optimizer, loss function, scheduler

# train, test after each epoch

#================

In [ ]:
# save model 
# torch.save(resnext50.state_dict(), "bird_class_model.pt")

# load model
# resnext50 = models.resnext50_32x4d()
# resnext50.fc = nn.Linear(2048, birddataset_train.n_classes)
# resnext50.load_state_dict(torch.load("bird_class_model.pt"))
# resnext50.eval()
# resnext50 = resnext50.to(device)
print("done!")

## Inspecting the results
Now that everything is working, lets look at some examples, you probably can't play the sounds in this notebook, so open them outside of the notebook (download them if needed) if you want to get a feel for what the model can and can't do.

In [ ]:
import matplotlib.pyplot as plt

def plot(image_batch, target_batch, fnames):
    image = image_batch[0]
    image2 = image_batch[1]
    target = target_batch[0]
    
    image = image.permute(1, 2, 0)
    image2 = image2.permute(1, 2, 0)
    image_batch = image_batch.to(device)
    output = resnext50(image_batch).cpu()
    image_batch = image_batch.cpu()
    
    pred = torch.argmax(output, dim=1)
    
    # we're plotting normalized images instead of the originals, but it gets the point across I think
    fig, axs = plt.subplots(1, 2, figsize=(20,10))
    axs[0].imshow(image.numpy())
    axs[0].set_title(f"class: {target_batch[0]}, predicted: {pred[0].item()},\n filepath: {fnames[0]}")
    axs[1].imshow(image2.numpy())
    axs[1].set_title(f"class: {target_batch[1]}, predicted: {pred[1].item()},\n filepath: {fnames[1]}")
    print(fnames[0], fnames[1])

for image_batch, target_batch, fnames in test_loader:
    plot(image_batch, target_batch, fnames) 
    break

## Outro
Well... you made it to the end of the exercises! Hope you enjoyed working on them and learned something ;). 
Some suggestions for follow up excersis:
- deep dream the various models that haven't been deepdreamed, you could even try deepdreaming the bird detector.
- try different backbones and see how large models actually have to be for the various problems, does it make a difference which one you use or are they pretty similar?
- try deep dreaming whole layers and try layers early or late into the model, is it also possible to optimize for two targets at very different locations in the model?